In [1]:
import sys
from pathlib import Path
import os 
sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')

current_path = Path.cwd()

# Go to top of the root and append
root = current_path.parents[3]
sys.path.append(str(root))

os.chdir(root)

In [2]:
import os
import json
from pathlib import Path

import openai

from fairness_cv_project.methods.label_free_cbm.src.utils import data_utils

In [20]:
dataset = "phoning_eating"
prompt_type = "around"

In [21]:
openai.api_key = open(os.path.join(os.path.expanduser("~"), ".openai_api_key"), "r").read()[:-1]

In [22]:
prompts = {
    "important" : "List the most important features for recognizing something as a \"goldfish\":\n\n-bright orange color\n-a small, round body\n-a long, flowing tail\n-a small mouth\n-orange fins\n\nList the most important features for recognizing something as a \"beerglass\":\n\n-a tall, cylindrical shape\n-clear or translucent color\n-opening at the top\n-a sturdy base\n-a handle\n\nList the most important features for recognizing something as a \"{}\":",
    "superclass" : "Give superclasses for the word \"tench\":\n\n-fish\n-vertebrate\n-animal\n\nGive superclasses for the word \"beer glass\":\n\n-glass\n-container\n-object\n\nGive superclasses for the word \"{}\":",
    "around" : "List the things most commonly seen around a \"tench\":\n\n- a pond\n-fish\n-a net\n-a rod\n-a reel\n-a hook\n-bait\n\nList the things most commonly seen around a \"beer glass\":\n\n- beer\n-a bar\n-a coaster\n-a napkin\n-a straw\n-a lime\n-a person\n\nList the things most commonly seen around a \"{}\":",
    "job": "List the things commonly seen around a \"teacher\":\n\n- a classroom\n-a desk\n-a chalkboard\n-a book\n-a student\n-a pencil\n-a pen\n\nList the things commonly seen around a \"software engineer\":\n\n- a computer\n-a desk\n-a keyboard\n-a mouse\n-a monitor\n-a chair\n-a person\n\nList the things commonly seen around someone's working as a \"{}\":",
}


base_prompt = prompts[prompt_type]

In [12]:
cls_file = data_utils.LABEL_FILES[dataset]
with open(root / cls_file, "r") as f:
    classes = f.read().split("\n")

In [23]:
feature_dict = {}

for i, label in enumerate(classes):
    feature_dict[label] = set()
    print("\n", i, label)
    for _ in range(2):
        response = openai.Completion.create(
              model="text-davinci-002",
              prompt=base_prompt.format(label),
              temperature=0.7,
              max_tokens=256,
              top_p=1,
              frequency_penalty=0,
              presence_penalty=0
            )
        #clean up responses
        print(response)
        features = response["choices"][0]["text"]
        features = features.split("\n-")
        features = [feat.replace("\n", "") for feat in features]
        features = [feat.strip() for feat in features]
        features = [feat for feat in features if len(feat)>0]
        features = set(features)
        feature_dict[label].update(features)
    feature_dict[label] = sorted(list(feature_dict[label]))


 0 eating
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n-food\n-a table\n-a chair\n-a napkin\n-a fork\n-a knife\n-a spoon\n-a person"
    }
  ],
  "created": 1688646440,
  "id": "cmpl-7ZIMqdVdps9aOU0TQpCZVRbl3Iquw",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 33,
    "prompt_tokens": 96,
    "total_tokens": 129
  }
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n- a kitchen\n-a table\n-a chair\n-a place setting\n-silverware\n-a glass\n-a plate\n-food"
    }
  ],
  "created": 1688646441,
  "id": "cmpl-7ZIMr5DjIxXdExR120QTYPQUlZyzN",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 33,
    "prompt_tokens": 96,
    "total_tokens": 129
  }
}

 1 phoning
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,


In [24]:
json_object = json.dumps(feature_dict, indent=4)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_{}_new.json".format(dataset, prompt_type), "w") as outfile:
    outfile.write(json_object)